# 번호판 인식기

## <br>라이브러리

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract

### <br>파일이름에 한글이 포함되어있음.<br>OpenCV에서 읽을 수 있도록 함

In [9]:
original_img_array = np.fromfile('C:/Users/admin/Downloads/test2.jpg', np.uint8)
original_img = cv2.imdecode(original_img_array, cv2.IMREAD_COLOR)
height, width, channel = original_img.shape
gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

## <br>오리지널 이미지 확인

In [10]:
cv2.imshow('Original Image', original_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>GRAY 이미지 확인

In [5]:
cv2.imshow('Gray Image', gray_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>노이즈 제거를 위한 Blur처리와 이미지 이진화

In [6]:
img_blurred = cv2.GaussianBlur(gray_img, ksize=(5,5), sigmaX=0)
img_thresh = cv2.adaptiveThreshold(img_blurred, maxValue=255.0,\
adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
thresholdType = cv2.THRESH_BINARY_INV, blockSize = 19, C = 9)

cv2.imshow('Thresh_image', img_thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>윤곽선(컨투어) 찾고 그리기

In [7]:
contours, _ = cv2.findContours(img_thresh, mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_SIMPLE)
temp_img = np.zeros((height, width, channel), dtype=np.uint8)
cv2.drawContours(temp_img, contours=contours, contourIdx=-1, color=(255, 255, 255))
cv2.imshow('Contour Image', temp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>사각형 그리기

In [11]:
all_contours_info = []    #각각의 contour의 x, y, w, h 등의 정보를 담아둘 곳
temp_img = np.zeros((height, width, channel), dtype=np.uint8)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(temp_img, pt1=(x,y), pt2=(x+w, y+h), color=(255, 255, 255), thickness=2)
    
    all_contours_info.append({'contour' : contour,
                             'x' : x,
                             'y' : y,
                             'w' : w,
                             'h' : h,
                             'cx' : x+(w/2),
                             'cy' : y+(h/2)})

cv2.imshow('Contour Image with Rect', temp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>번호판 글자만 추출
### <br>1. 크기 조건으로 판단

In [15]:
MIN_AREA = 80                       # 최소 넓이
MIN_WIDTH, MIN_HEIGHT = 2, 8        # 최소 너비, 높이
MIN_RATIO, MAX_RATIO = 0.25, 1.0    # 최소 비율 (가로 대비 세로 비율)

size_ok_contours = []
count = 0
for contour_info in all_contours_info:
    area = contour_info['w'] * contour_info['h']
    ratio = contour_info['w'] / contour_info['h']
    
    if area > MIN_AREA and contour_info['w'] > MIN_WIDTH \
    and contour_info['h'] > MIN_HEIGHT and MIN_RATIO < ratio < MAX_RATIO:
        contour_info['index'] = count
        size_ok_contours.append(contour_info)
        count += 1

temp_img = np.zeros((height, width, channel), dtype=np.uint8)
for contour in size_ok_contours:
    cv2.rectangle(temp_img,
    pt1 = (contour['x'], contour['y']),
    pt2 = (contour['x'] + contour['w'], contour['y'] + contour['h']),
    color = (255, 255, 255), thickness = 2)

cv2.imshow('selected_rect', temp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### <br>2. 정렬방식으로 판단

In [16]:
MAX_DIAGONAL_SCALE = 5    # 사각형의 대각선 길이의 최대 5배가 최대 간격
MAX_ANGLE_DIFF = 12.0     # 사각형의 중심 최대 각도
MAX_AREA_DIFF = 0.5       # 사각형의 최대 면적 차이
MAX_WIDTH_DIFF, MAX_HEIGHT_DIFF = 0.8, 0.2 # 가로, 세로 차이
MIN_MATCHED_NUM = 3       # 조건을 만족하는 사각형의 최소 개수

def findrect(possible_contours): # possible_contours = size_ok_contours
    final_matched_index = []     # 최종 후보군 인덱스 모음
    # 이중 for문으로 contour 사각형 2개를 비교할 것임.
    for rect1 in possible_contours:
        matched_contours_index = [] # MIN_MATCHED_NUM을 제외한 모든 조건을 통과한 후보군 인덱스 모음
        for rect2 in possible_contours:
            if rect1['index'] == rect2['index']: #같은 contour는 같은 index이므로 pass
                continue
            # 첫 번째 contour 사격형의 대각선 길이. sqrt(): 루트근사값
            rect1_diagonal_length = np.sqrt(rect1['w']**2 + rect1['h']**2) # a^2 + b^2 = c^2
            # np.linalg.norm(a-b): 벡터 a와 벡터 b사이의 거리를 구함 - 두 contour 중심점 사이 길이
            distance = np.linalg.norm(np.array([rect1['cx'], rect1['cy']]) - np.array([rect2['cx'], rect2['cy']]))
            # dx: 직삼각형 만들 때의 밑변 / dy: 직삼각형 만들 때의 세로변
            dx = abs(rect1['cx']-rect2['cx'])
            dy = abs(rect1['cy']-rect2['cy'])
            if dx == 0: # dx가 0이면 서로 같은 x좌표이기 때문에 예외처리로 90도로 설정
                angle_diff = 90
            else:
                angle_diff = np.degrees(np.arctan(dy / dx))
                
            area_diff = abs(rect1['w']*rect1['h'] - rect2['w']*rect2['h']) / (rect1['w']*rect1['h'])
            width_diff = abs(rect1['w'] - rect2['w']) / rect1['w'] # 가로의 비율
            height_diff = abs(rect1['h'] - rect2['h']) / rect1['h'] # 높이의 비율
            
            if distance < rect1_diagonal_length * MAX_DIAGONAL_SCALE \
                and angle_diff < MAX_ANGLE_DIFF \
                and area_diff < MAX_AREA_DIFF and width_diff < MAX_WIDTH_DIFF and height_diff < MAX_HEIGHT_DIFF:
                matched_contours_index.append(rect2['index'])
        matched_contours_index.append(rect1['index']) # rect1은 첫번째 for문에서
        
        if len(matched_contours_index) < MIN_MATCHED_NUM:
            continue
        final_matched_index.append(matched_contours_index)
        
        unmatched_contours_index = [] # 후보군에 들지 못한 비후보군 인덱스 모음
        for rect3 in possible_contours:
            if rect3['index'] not in matched_contours_index:
                unmatched_contours_index.append(rect3['index'])
        # np.take(a, b): a에서 b값 인덱스들을 추출.
        unmatched_contours = np.take(size_ok_contours, unmatched_contours_index)
        
        recursive_contours_index = findrect(unmatched_contours)
        for index in recursive_contours_index:
            final_matched_index.append(index)
            break
    return final_matched_index

result_index = findrect(size_ok_contours) # 최종 후보군 index를 통해 저장
result_contours = [] # 최종 후보군 index를 통해 얻을 contour 정보 저장하기 위하여
for index in result_index:
    # 최종 후보군 index 정보로 size_ok_contours에서 최종 contour 선별
    result_contours.append(np.take(size_ok_contours, index))

## <br>최종 후보군에 들어있는 contour를 시각화

In [18]:
temp_img = np.zeros((height, width, channel), dtype=np.uint8)
for contour in result_contours: # 최종 후보군 시각화
    for rect in contour:
        cv2.rectangle(temp_img, pt1=(rect['x'], rect['y']), pt2=(rect['x']+rect['w'], rect['y']+rect['h']), color=(255,255,255), thickness=2)
cv2.imshow('Visualization', temp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>기울어진 번호판 수평으로 맞추기(리턴된 각도만큼 이미지 회전 부분 추가해야함)

In [19]:
PLATE_WIDTH_PADDING = 1.3
PLATE_HEIGHT_PADDING = 1.5
MIN_PLATE_RATIO = 3
MAX_PLATE_RATIO = 10

for i, result_contour in enumerate(result_contours):
    # sorted_contour: 정렬된 최종 후보군 contour들
    sorted_contour = sorted(result_contour, key=lambda x: x['cx']) # x방향으로 순차적 정렬
    
    # 첫번째 contour와 마지막(-1) contour로 처음과 끝 중간좌표 구하기
    plate_cx = (sorted_contour[0]['cx'] + sorted_contour[-1]['cx']) / 2
    plate_cy = (sorted_contour[0]['cy'] + sorted_contour[-1]['cy']) / 2
    plate_width = (sorted_contour[-1]['x'] + sorted_contour[-1]['w'] - sorted_contour[0]['x']) * PLATE_WIDTH_PADDING
    sum_height = 0
    for rect in sorted_contour:
        sum_height += rect['h']
    plate_height = int(sum_height / len(sorted_contour) * PLATE_HEIGHT_PADDING)
    triangle_height = sorted_contour[-1]['cy'] - sorted_contour[0]['cy']
    triangle_hypotenus = np.linalg.norm(np.array([sorted_contour[0]['cx'], sorted_contour[0]['cy']]) - np.array([sorted_contour[-1]['cx'], sorted_contour[-1]['cy']]))\
    # arcsin을 이용해 각도를 구한다. (빗변과 세로변이므로 tan가 아닌 sin)
    angle = np.degrees(np.arcsin(triangle_height / triangle_hypotenus))
    
cv2.imshow('Horizantalization', temp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
angle

1.3019526725788753